# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,st. john's,47.5615,-52.7126,4.48,98,100,3.09,CA,2024-05-03 04:46:05
1,1,waingapu,-9.6522,120.2637,29.59,65,20,7.16,ID,2024-05-03 04:48:45
2,2,lagoa,37.1326,-8.4551,11.71,77,17,2.36,PT,2024-05-03 04:48:45
3,3,vung tau,10.3486,107.0765,30.26,67,43,5.61,VN,2024-05-03 04:48:46
4,4,uliastay,47.7416,96.8475,10.95,29,5,1.60,MN,2024-05-03 04:48:47


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
# %%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points(
    x = "Lng",
    y = "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    color = "City"
)

# Define the size of each point based on humidity
import warnings
warnings.filterwarnings("ignore")

# Display the map

city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df2 = city_data_df.copy()
city_data_df2 = city_data_df2[(city_data_df2['Max Temp'] > 20) & (city_data_df['Max Temp'] < 35)]
city_data_df2 = city_data_df2[city_data_df2['Cloudiness'] < 10]
city_data_df2 = city_data_df2[city_data_df2['Humidity'] < 30]
city_data_df2 = city_data_df2[city_data_df2['Wind Speed'] < 6]

# Drop any rows with null values
city_data_df2 = city_data_df2.dropna()

# Display sample data
city_data_df2

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
26,26,korla,41.7239,86.1735,23.12,12,0,3.94,CN,2024-05-03 04:49:03
120,120,shendi,16.6885,33.4323,27.78,12,1,4.31,SD,2024-05-03 04:50:18
121,121,al jawf,24.1985,23.2939,26.46,24,4,2.17,LY,2024-05-03 04:50:18
167,167,crane,31.3957,-102.3521,25.23,19,0,2.98,US,2024-05-03 04:50:55
227,227,at taj,24.2173,23.2987,26.46,24,4,2.17,LY,2024-05-03 04:51:44
235,235,alice springs,-23.6984,133.8813,21.75,26,0,5.14,AU,2024-05-03 04:51:50
236,236,al kharijah,15.3595,44.2167,20.02,20,5,1.61,YE,2024-05-03 04:51:51
245,245,chuguyevka,44.1687,133.8730,20.32,17,3,5.60,RU,2024-05-03 04:51:58
305,305,carnarvon,-24.8826,113.6576,30.50,27,0,2.07,AU,2024-05-03 04:52:44
312,312,pecos,31.4204,-103.4975,26.58,9,0,4.63,US,2024-05-03 04:52:50


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

hotel_df = city_data_df2.copy()
kept_columns = ["City", "Country", "Lat", "Lng", "Humidity"]
dropped_columns = []
for column in list(hotel_df.columns):
    if column not in kept_columns:
        dropped_columns.append(column)
hotel_df = hotel_df.drop(columns=dropped_columns)

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = [""] * len(hotel_df)

# Display sample data
hotel_df

,City,Lat,Lng,Humidity,Country,Hotel Name
26,korla,41.7239,86.1735,12,CN,
120,shendi,16.6885,33.4323,12,SD,
121,al jawf,24.1985,23.2939,24,LY,
167,crane,31.3957,-102.3521,19,US,
227,at taj,24.2173,23.2987,24,LY,
235,alice springs,-23.6984,133.8813,26,AU,
236,al kharijah,15.3595,44.2167,20,YE,
245,chuguyevka,44.1687,133.8730,17,RU,
305,carnarvon,-24.8826,113.6576,27,AU,
312,pecos,31.4204,-103.4975,9,US,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 100_000
params = {
    "categories": "accommodation.hotel",
    "lang": "en",
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    headers = {
        "Accept": "application/json"
    }
    name_address = requests.get(url = base_url, headers = headers, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
korla - nearest hotel: 庫爾勒龍行青年旅舍
shendi - nearest hotel: Local hotel guesthouse
al jawf - nearest hotel: No hotel found
crane - nearest hotel: No hotel found
at taj - nearest hotel: No hotel found
alice springs - nearest hotel: Todd Tavern
al kharijah - nearest hotel: برج السلام
chuguyevka - nearest hotel: No hotel found
carnarvon - nearest hotel: No hotel found
pecos - nearest hotel: Holiday Inn Express & Suites Pecos
hohhot - nearest hotel: 内蒙古国航大厦
sao vicente - nearest hotel: Pousada Vitória
hayma' - nearest hotel: فندق أبراج المطار
aswan - nearest hotel: El-Salam Hotel
joshimath - nearest hotel: Snow Crest Hotel


,City,Lat,Lng,Humidity,Country,Hotel Name
26,korla,41.7239,86.1735,12,CN,庫爾勒龍行青年旅舍
120,shendi,16.6885,33.4323,12,SD,Local hotel guesthouse
121,al jawf,24.1985,23.2939,24,LY,No hotel found
167,crane,31.3957,-102.3521,19,US,No hotel found
227,at taj,24.2173,23.2987,24,LY,No hotel found
235,alice springs,-23.6984,133.8813,26,AU,Todd Tavern
236,al kharijah,15.3595,44.2167,20,YE,برج السلام
245,chuguyevka,44.1687,133.8730,17,RU,No hotel found
305,carnarvon,-24.8826,113.6576,27,AU,No hotel found
312,pecos,31.4204,-103.4975,9,US,Holiday Inn Express & Suites Pecos


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
# %%capture --no-display

# Configure the map plot

hotel_map = hotel_df.hvplot.points(
    x = "Lng",
    y = "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    color = "Country",
    hover_cols = ['Hotel Name', 'Country']
)

# Display the map

hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Country,Hotel Name)